In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# **民族服饰分类**

In [2]:
import matplotlib.pyplot as plt

In [3]:
import os
import cv2
bad_list=[]
dir=r'./images'
subdir_list=os.listdir(dir) # create a list of the sub directories in the directory ie train or test
for d in subdir_list:  # iterate through the sub directories train and test
    dpath=os.path.join (dir, d) # create path to sub directory
    if d in ['test', 'train']:
        class_list=os.listdir(dpath) # list of classes ie dog or cat
       # print (class_list)
        for klass in class_list: # iterate through the two classes
            class_path=os.path.join(dpath, klass) # path to class directory
            #print(class_path)
            if class_path == dpath+'/.DS_Store':
                os.remove(class_path)
                continue
            file_list=os.listdir(class_path) # create list of files in class directory
            for f in file_list: # iterate through the files
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') # find index of period infilename
                ext=f[index+1:] # get the files extension
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    os.remove(fpath)
                    print("Successfully removed!")
                    bad_list.append(fpath)
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        os.remove(fpath)
                        print("Successfully removed!")
                        bad_list.append(fpath)

print (bad_list)

Premature end of JPEG file


[]


In [3]:
from keras.preprocessing.image import ImageDataGenerator

shift = 0.3
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.5,
    zoom_range=0.2,
    rotation_range=30,
    width_shift_range=shift, height_shift_range=shift,
    featurewise_center=True,
    # featurewise_std_normalization=True,
    zca_whitening=True,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [4]:
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
batch_size = 25

train_generator = train_datagen.flow_from_directory(
    # './images_changed/train',
    './images/train',
    # target_size=(300, 300),
    batch_size=batch_size,
    # class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    # './images_changed/test',
    './images/test',
    # target_size=(150, 150),
    batch_size=batch_size,
    # class_mode='binary'
)

Found 2607 images belonging to 56 classes.
Found 1105 images belonging to 56 classes.


In [5]:
from keras.applications.densenet import DenseNet121

In [6]:
"""
ResNet-18
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""

from keras import layers
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Add, Activation, ZeroPadding2D, Dropout, AveragePooling2D
from keras.models import Model

class Conv(Model):

    def __init__(self, channels: int):

        super().__init__()

        self.conv = Conv2D(channels,kernel_size=3,use_bias=False)

    def call(self, input):

        return self.conv(input)

class dense_block(Model):

    def __init__(self,nb_layers: int, nb_filters: int, dropout_rate=None, grop_rate=0):

        super().__init__()

        self.__nb_layers = nb_layers
        self.__channels = nb_filters
        self.__dropout_rate = dropout_rate
        self.__grop_rate = grop_rate

        self.zero_pool = ZeroPadding2D()
        self.conv = Conv2D(self.__channels,kernel_size=3,use_bias=False)
        self.bn = BatchNormalization()
        self.drop_out = Dropout(self.__dropout_rate)

    def call(self, input):
        concat_feat = input

        for _ in range(self.__nb_layers):
            x = self.zero_pool(input)
            x = self.conv(x)
            x = Activation('relu')(x)
            x = self.bn(x)

            if self.__dropout_rate:
                x = self.drop_out(x)

            concat_feat =  layers.Concatenate()([x,concat_feat])

            if self.__grop_rate!=0:
                self.__channels += self.__grop_rate

        return concat_feat

class transition_block(Model):

    def __init__(self, channels: int):

        super().__init__()

        self.__channels = channels

        self.bn_1 = BatchNormalization()
        self.conv = Conv2D(self.__channels,1,1,use_bias=False)
        self.avg = AveragePooling2D(2,2)

    def call(self, input):

        x = self.bn_1(input)
        x = Activation('relu')(x)
        x = self.conv(x)
        out = self.avg(x)

        return out

class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = Activation('relu')(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = Activation('relu')(x)
        return out

class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)

        self.den_1_1 = dense_block(nb_layers=4,nb_filters=64,dropout_rate=0.2,grop_rate=32)
        self.trans_1 = transition_block(256)
        self.den_1_2 = dense_block(nb_layers=4,nb_filters=128,dropout_rate=0.2,grop_rate=32)
        self.trans_2 = transition_block(512)

        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = Activation('relu')(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)

        out = self.den_1_1(out)
        out = self.trans_1(out)

        #第二个块
        out = self.den_1_2(out)
        out = self.trans_2(out)

        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [7]:
model = ResNet18(56)
# model = DenseNet121(
#     classes=56,
#     weights=None,
#     input_shape=(256, 256, 3),
#     # pooling=None,
#     # name="DenseNet121",
# )
model.build(input_shape = (None,)+train_generator.image_shape)
#use categorical_crossentropy since the label is one-hot encoded
from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

Metal device set to: AMD Radeon Pro 5500M

systemMemory: 16.00 GB
maxCacheSize: 3.99 GB



2023-03-09 21:06:55.848598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-09 21:06:55.850114: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  9472      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                     

In [8]:
from keras.callbacks import EarlyStopping
import scipy

es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_accuracy")
#I did not use cross validation, so the validate performance is not accurate.
# STEPS = len(X_train) / batch_size
history = model.fit(train_generator, epochs=150, validation_data=validation_generator,
                    # callbacks=[es]
                    )

/Users/precious/opt/anaconda3/envs/tensorflow_metal/lib/python3.8/site-packages/keras/preprocessing/image.py:1663: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/precious/opt/anaconda3/envs/tensorflow_metal/lib/python3.8/site-packages/keras/preprocessing/image.py:1682: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/150


2023-03-09 21:07:32.286703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 18/105 [====>.........................] - ETA: 2:12 - loss: 4.6383 - accuracy: 0.0178

KeyboardInterrupt: 

In [ ]:
# 借助 history 对象了解训练过程
history_dict = history.history
acc = history_dict['accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
val_accuracy = history_dict['val_accuracy']

# 借助 Matplotlib 绘制图像
plt.plot(range(1, len(acc)+1), acc, 'b--')
plt.plot(range(1, len(loss)+1), loss, 'r-')
# plt.plot(range(1, len(val_loss)+1), val_loss, '-')
plt.plot(range(1, len(val_accuracy)+1), val_accuracy, '--')

# 显示图例
plt.legend(['accuracy', 'loss'
               , 'val_accuracy'])